<img src="https://afs-services.com/wp-content/uploads/2025/09/pypricing_banner.png" alt="RV Banner" style="width: 100%; height: auto; display: block;">

# Comparison CDS default and premium leg

Old code vs New code. We assume the same survival curve for both.

# Import

In [1]:
import sys, os, json
import numpy as np

np.set_printoptions(precision=50, suppress=True)
import pandas as pd

repo_root = os.path.abspath("..")
sys.path.insert(0, repo_root)
import afslibrary as afs
from pathlib import Path

import db_tools

beautiful_data = db_tools.BeautifulDataAFSStyle()
d = afs.DataFactory(beautiful_data)
calendars = d.import_calendar("Act360")

# Example 1 (similar results)

CDS Survival Curve

In [2]:
evaluation_date = pd.to_datetime("2020-06-20")
maturity = pd.to_datetime("2022-06-20")

We check they are IMM dates

In [3]:
afs.CDSCurve.get_imm_dates(evaluation_date, evaluation_date)

array([Timestamp('2020-06-22 00:00:00')], dtype=object)

In [4]:
afs.CDSCurve.get_imm_dates(maturity, maturity)

array([Timestamp('2022-06-20 00:00:00')], dtype=object)

In [5]:
risk_free_r = 0.02
recovery_rate = 0.4

discount_curve = afs.CRDC(r=risk_free_r, calendar=calendars["Act360"])
tenors = [1, 2, 4, 8]  # In years
hazard_rates = [0.02, 0.03, 0.04, 0.09]

cds_curve = afs.CDSCurve(
    "CDS Survival", True, calendar=calendars["Act360"], discount_curve=discount_curve
)

arr = np.array(hazard_rates)
arr_2d = arr.reshape(1, -1)
cols = tenors
df_hazard_rates = pd.DataFrame(arr_2d, index=[evaluation_date], columns=cols)

Equivalent to the result of CDSCurve.fit

In [6]:
cds_curve.hazard_rates = df_hazard_rates
cds_curve.fitting_dates = df_hazard_rates.index
cds_curve.interpolation_data = -df_hazard_rates
cds_curve.params = {
    -1: -df_hazard_rates,
    0: -df_hazard_rates,
    1: -df_hazard_rates,
}  # For this example we do not consider the perturbations.

CDS legs (CRS, using the CDS class with the new modifications)

In [7]:
nominal = 1000000.0
premium_rate = 0.05

cds_pypricing = afs.CDS(
    cds_curve,
    effective_date=evaluation_date,
    maturity=maturity,
    premium_rate=premium_rate,
    nominal=nominal,
)
print(
    cds_pypricing.get_px(dates=evaluation_date, discount_curve=discount_curve)
)  # clean=True since t is a payment date


Default leg: [29241.06430195958]
2020-06-20   -67373.987084
dtype: float64


Artur (now using the old CDSRate class)

In [8]:
cds_afa = afs.CDSRate(
    cds_curve,
    effective_date=evaluation_date,
    end_date=maturity,
    freq=3,
    calendar=calendars["Act360"],
)
print(
    f"Coupon/Premium Leg: {-premium_rate * cds_afa.get_rpvbp(dates=evaluation_date, discount_curve=discount_curve)}"
)
print(
    f"Protection/Default Leg: {cds_afa.get_value(dates=evaluation_date, discount_curve=discount_curve) * cds_afa.get_rpvbp(dates=evaluation_date, discount_curve=discount_curve)}"
)

Coupon/Premium Leg: 2020-06-20   -0.096892
dtype: float64
Protection/Default Leg: 2020-06-20    0.02924
dtype: float64


# Example 2 (different results due to the definition of maturity)

In these case the results are different due to the definition of maturity in a CDS contract.

CDS Survival Curve

In [9]:
evaluation_date = pd.to_datetime("2023-03-20")
maturity = pd.to_datetime("2024-06-21")

Note that maturity  is calculated as the first ‘IMM’ date T years after the effective date. In consequence, if the maturity introduced is not a IMM date,
      the next IMM date is considered.

In [10]:
afs.CDSCurve.get_imm_dates(maturity, maturity)

array([Timestamp('2024-09-20 00:00:00')], dtype=object)

In [11]:
risk_free_r = 0.04
recovery_rate = 0.2

discount_curve = afs.CRDC(r=risk_free_r, calendar=calendars["Act360"])
tenors = [0.5, 2, 4, 6]  # In years
hazard_rates = [0.0250, 0.0325, 0.0452, 0.0865]

cds_curve = afs.CDSCurve(
    "CDS Survival", True, calendar=calendars["Act360"], discount_curve=discount_curve
)

arr = np.array(hazard_rates)
arr_2d = arr.reshape(1, -1)
cols = tenors
df_hazard_rates = pd.DataFrame(arr_2d, index=[evaluation_date], columns=cols)

Equivalent to the result of CDSCurve.fit

In [12]:
cds_curve.hazard_rates = df_hazard_rates
cds_curve.fitting_dates = df_hazard_rates.index
cds_curve.interpolation_data = -df_hazard_rates
cds_curve.params = {
    -1: -df_hazard_rates,
    0: -df_hazard_rates,
    1: -df_hazard_rates,
}  # For this example we do not consider the perturbations.

CDS legs (CRS, using the CDS class with the new modifications)

In [13]:
nominal = 1000000.0
premium_rate = 0.05

cds_pypricing = afs.CDS(
    cds_curve,
    effective_date=evaluation_date,
    maturity=maturity,
    premium_rate=premium_rate,
    nominal=nominal,
)
print(
    cds_pypricing.get_px(dates=evaluation_date, discount_curve=discount_curve)
)  # clean=True since t is a payment date


Default leg: [26075.106468046193]
2023-03-20   -46091.045211
dtype: float64


Artur (now using the old CDSRate class)

In [14]:
cds_afa = afs.CDSRate(
    cds_curve,
    effective_date=evaluation_date,
    end_date=maturity,
    freq=3,
    calendar=calendars["Act360"],
)
print(
    f"Coupon/Premium Leg: {-premium_rate * cds_afa.get_rpvbp(dates=evaluation_date, discount_curve=discount_curve)}"
)
print(
    f"Protection/Default Leg: {cds_afa.get_value(dates=evaluation_date, discount_curve=discount_curve) * cds_afa.get_rpvbp(dates=evaluation_date, discount_curve=discount_curve)}"
)

Coupon/Premium Leg: 2023-03-20   -0.060764
dtype: float64
Protection/Default Leg: 2023-03-20    0.021604
dtype: float64
